# Smart Cultural Storyteller - AI Content Generation

This notebook generates images and videos for the Betal storytelling project using free AI models.

## Setup and Dependencies

In [3]:
# Install required packages
%pip install requests pillow opencv-python moviepy huggingface-hub transformers torch diffusers
%pip install gTTS  
%pip install gradio python-dotenv
!

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\itske\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\itske\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\itske\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import requests
import json
import os
import time
from PIL import Image, ImageEnhance, ImageFilter
import cv2
import numpy as np
from moviepy.editor import *
#import torch
#from diffusers import StableDiffusionPipeline
from gtts import gTTS
import io
from pathlib import Path

## Configuration

In [2]:
# API Keys
from dotenv import load_dotenv
import os

load_dotenv()  # take variables from .env
HUGGINGFACE_TOKEN = os.getenv("HUGGINGFACE_TOKEN")


# Output directories
OUTPUT_DIR = Path("public/generated_content")
IMAGES_DIR = OUTPUT_DIR / "images"
AUDIO_DIR = OUTPUT_DIR / "audio"
VIDEOS_DIR = OUTPUT_DIR / "videos"

# Create directories
for dir_path in [IMAGES_DIR, AUDIO_DIR, VIDEOS_DIR]:
    dir_path.mkdir(parents=True, exist_ok=True)

## Story Data

## Image Generation using Stable Diffusion

In [3]:
def check_model_status(model_name):
    """Check if the model is loaded and ready"""
    API_URL = f"https://api-inference.huggingface.co/models/{model_name}"
    headers = {"Authorization": f"Bearer {HUGGINGFACE_TOKEN}"}
    
    response = requests.get(API_URL, headers=headers)
    print(f"Model status: {response.status_code}")
    if response.status_code == 200:
        return True
    return False

def generate_image_hf_api(prompt, filename, max_retries=3):
    """Generate image using Hugging Face Inference API with better error handling"""
    
    # Try different models if one fails
    models = [
        "black-forest-labs/FLUX.1-schnell"
    ]
    
    for model in models:
        API_URL = f"https://api-inference.huggingface.co/models/{model}"
        headers = {"Authorization": f"Bearer {HUGGINGFACE_TOKEN}"}
        
        # Enhanced prompt for Indian cultural context
        enhanced_prompt = f"{prompt}, Indian cultural motifs, traditional art style, highly detailed, rich textures, vivid harmonious colors, intricate patterns, cinematic soft lighting, emotional storytelling, high quality"
        
        payload = {
            "inputs": enhanced_prompt,
            "parameters": {
                "guidance_scale": 7.5,
                "num_inference_steps": 50,
                "width": 512,
                "height": 512
            }
        }
        
        for attempt in range(max_retries):
            try:
                print(f"Attempting to generate: {prompt[:50]}... (Model: {model}, Attempt: {attempt+1})")
                
                response = requests.post(API_URL, headers=headers, json=payload, timeout=60)
                
                if response.status_code == 200:
                    # Check if response is actually an image
                    if response.headers.get('content-type', '').startswith('image/'):
                        image_path = IMAGES_DIR / f"{filename}.png"
                        with open(image_path, "wb") as f:
                            f.write(response.content)
                        print(f"✅ Generated image: {image_path}")
                        return image_path
                    else:
                        print(f"❌ Response is not an image. Content-Type: {response.headers.get('content-type')}")
                        try:
                            error_data = response.json()
                            print(f"Error response: {error_data}")
                        except:
                            print(f"Raw response: {response.text[:200]}...")
                
                elif response.status_code == 503:
                    try:
                        error_data = response.json()
                        if "loading" in str(error_data).lower():
                            estimated_time = error_data.get("estimated_time", 30)
                            print(f"⏳ Model is loading. Waiting {estimated_time} seconds...")
                            time.sleep(estimated_time + 10)  # Wait a bit longer
                            continue
                    except:
                        pass
                    print(f"❌ Service unavailable: {response.text}")
                
                else:
                    print(f"❌ Error {response.status_code}: {response.text}")
                
                # Wait before retry
                if attempt < max_retries - 1:
                    wait_time = (attempt + 1) * 5
                    print(f"⏳ Waiting {wait_time} seconds before retry...")
                    time.sleep(wait_time)
                    
            except requests.exceptions.Timeout:
                print(f"⏳ Request timeout. Retrying...")
                time.sleep(10)
            except Exception as e:
                print(f"❌ Exception: {str(e)}")
                time.sleep(5)
        
        print(f"❌ Failed with model {model}, trying next model...")
        time.sleep(5)
    
    print("❌ All models failed")
    return None

def test_api_connection():
    """Test the API connection and token validity"""
    headers = {"Authorization": f"Bearer {HUGGINGFACE_TOKEN}"}
    test_url = "https://api-inference.huggingface.co/models/runwayml/stable-diffusion-v1-5"
    
    try:
        response = requests.get(test_url, headers=headers)
        print(f"API Test - Status Code: {response.status_code}")
        if response.status_code == 401:
            print("❌ Invalid or missing token. Please check your HUGGINGFACE_TOKEN")
            return False
        elif response.status_code == 200:
            print("✅ API connection successful")
            return True
        else:
            print(f"⚠️ Unexpected response: {response.text}")
            return True  # Might still work
    except Exception as e:
        print(f"❌ API test failed: {str(e)}")
        return False

In [4]:
def generate_story_images(story_data):
    """Generate images for the story with better error handling"""
    
    # Test API connection first
    if not test_api_connection():
        print("❌ API connection test failed. Please check your token.")
        return []
    
    image_paths = []
    
    for i, scene_prompt in enumerate(story_data["scenes"]):
        print(f"\n🎨 Generating image {i+1}/{len(story_data['scenes'])}")
        filename = f"{story_data['id']}_scene_{i+1:02d}"
        image_path = generate_image_hf_api(scene_prompt, filename)
        
        if image_path:
            image_paths.append(str(image_path))
        else:
            print(f"❌ Failed to generate image for scene {i+1}")
        
        # Add delay to avoid rate limiting
        print("⏳ Waiting to avoid rate limiting...")
        time.sleep(10)  # Increased delay
    
    return image_paths


## Audio Generation (Text-to-Speech)

In [5]:

def generate_audio_gtts(text, filename, lang='en', tld='co.in'):
    """Generate audio using Google Text-to-Speech with Indian accent (male-sounding)"""
    tts = gTTS(text=text, lang=lang, tld=tld, slow=True)  # Slower speech for deeper tone
    audio_path = AUDIO_DIR / f"{filename}.mp3"
    tts.save(str(audio_path))
    print(f"Generated audio: {audio_path}")
    return audio_path



## Video Generation with Effects

In [6]:
from pathlib import Path
import os, re
import numpy as np
from moviepy.editor import (
    ImageClip,
    CompositeVideoClip,
    concatenate_videoclips,
    AudioFileClip,
)

def _safe_name(name: str) -> str:
    base = os.path.splitext(os.path.basename(str(name)))[0]
    return re.sub(r'[^A-Za-z0-9._-]+', '_', base)[:80]

def ease_in_out_cubic(x: float) -> float:
    return 4 * x * x * x if x < 0.5 else 1 - (-2 * x + 2) ** 3 / 2

import cv2
import numpy as np
import moviepy.editor as mp
def get_blurred_background_cv2(img_path, out_size, blur_ksize=51):
    img = cv2.imread(img_path)
    img = cv2.resize(img, out_size)
    img = cv2.GaussianBlur(img, (blur_ksize, blur_ksize), 0)
    tmp_path = "temp_blur.jpg"
    cv2.imwrite(tmp_path, img)
    return mp.ImageClip(tmp_path).set_duration(5)


def add_parallax_effect(image_path, duration=5, out_size=(1280, 720), variant_idx=0, zoom=0.04, pan_px=30):
    from moviepy.editor import ImageClip, CompositeVideoClip, vfx
    import numpy as np

    def ease_in_out_cubic(x):
        return 4*x*x*x if x < 0.5 else 1 - (-2*x + 2) ** 3 / 2

    base = ImageClip(image_path).set_duration(duration)
    W, H = out_size
    img_w, img_h = base.size

    # scale image to fit inside frame (letterbox style)
    fit_scale = min(W / img_w, H / img_h)
    scale_start = fit_scale
    scale_end   = fit_scale * (1.0 + max(0.0, zoom))

    def scale_fn(t):
        p = ease_in_out_cubic(t / duration)
        return scale_start + (scale_end - scale_start) * p

    rng = np.random.RandomState(variant_idx or 0)
    pan_x = int(rng.choice([-1, 1]) * rng.randint(max(12, pan_px // 2), pan_px))
    pan_y = int(rng.choice([-1, 1]) * rng.randint(max(8, pan_px // 3), max(14, pan_px)))

    def pos_fn(t):
        p = ease_in_out_cubic(t / duration)
        # center alignment + parallax pan
        return (
            (W - img_w * scale_fn(t)) / 2 - pan_x * (1 - p),
            (H - img_h * scale_fn(t)) / 2 - pan_y * (1 - p),
        )

    # Foreground (main image, centered and animated)
    moving = base.resize(lambda t: scale_fn(t))

    # Background (blurred, scaled to cover full frame)
    background = get_blurred_background_cv2(image_path, out_size)


    return CompositeVideoClip(
        [background, moving.set_position(pos_fn)],
        size=out_size
    ).set_duration(duration)


def create_story_video(
    image_paths,
    audio_path,
    story_id,
    out_size=(1280, 720),
    fps=24,
    crossfade=0.3,       # set 0.0 to disable zoom entirely        # set 0 to disable pan entirely
    videos_dir=None,
):
    # Output directory
    if videos_dir is None:
        try:
            output_dir = Path(VIDEOS_DIR)
        except NameError:
            output_dir = Path("generated_content") / "videos"
    else:
        output_dir = Path(videos_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    # Audio
    if not (audio_path and os.path.exists(str(audio_path))):
        print("Audio file missing; provide valid audio_path.")
        return None
    audio_clip = AudioFileClip(str(audio_path))
    total_duration = audio_clip.duration

    n = len(image_paths)
    if n == 0:
        print("No images provided.")
        return None

    per = total_duration / n

    clips = []
    for i, img_path in enumerate(image_paths):
        if not os.path.exists(img_path):
            print(f"Image not found: {img_path}")
            continue

        start = i * per
        dur = per if i < n - 1 else max(0.1, total_duration - start)

        clip = add_parallax_effect(
            img_path,
            duration=dur,
            out_size=out_size,
            variant_idx=i
        )
        clip = clip.set_start(start)
        if i > 0 and crossfade > 0:
            clip = clip.crossfadein(min(crossfade, dur / 2))
        clips.append(clip)

    if not clips:
        print("No valid clips to compose.")
        return None

    timeline = concatenate_videoclips(clips, method="compose", padding=0)
    timeline = timeline.set_audio(audio_clip).set_fps(fps)

    safe_id = _safe_name(story_id)
    video_path = output_dir / f"{safe_id}_video.mp4"
    timeline.write_videofile(
        str(video_path),
        codec="libx264",
        audio=True,
        audio_codec="aac",
        fps=fps,
        remove_temp=True,
    )
    return str(video_path)

## Generate Subtitles

In [7]:
def generate_subtitles(story_content, audio_duration):
    """Generate subtitle timing for the story"""
    sentences = story_content.replace('\n', ' ').split('. ')
    sentences = [s.strip() + '.' for s in sentences if s.strip()]
    
    duration_per_sentence = audio_duration / len(sentences)
    
    subtitles = []
    current_time = 0
    
    for sentence in sentences:
        subtitle = {
            "start": round(current_time, 1),
            "end": round(current_time + duration_per_sentence, 1),
            "text": sentence
        }
        subtitles.append(subtitle)
        current_time += duration_per_sentence
    
    return subtitles



## Export Data for Web Application

## Batch Process All Stories

In [7]:
# Function to process all 10 stories
def process_all_stories():
    """Process all stories in the dataset"""
    
    # You can define all 10 stories here or load from external file
    stories = [
  {
    "id": "wisdom-1",
    "title": "The Monkey and the Crocodile",
    "theme": "wisdom",
    "content": "A clever monkey lived on a jamun tree by a river. One day, a crocodile tried to trick the monkey into coming onto his back to eat him. Using his wit, the monkey told the crocodile he had left his heart on the tree and must return to fetch it. The crocodile, fooled, brought him back to safety. The monkey's intelligence saved his life, proving that sharp thinking can triumph over brute strength.",
    "scenes": [
      "Riverbank view: the crocodile lurking beneath shimmering water, sun glinting on scales, monkey perched nervously on a jamun branch above",
      "Close encounter: monkey conversing with crocodile, subtle smirk, tension in the posture, shaded river reflections",
      "The clever trick: monkey pointing at a distant tree, expression animated, crocodile turning to look, sunlight catching ripples",
      "Escape sequence: monkey leaping from crocodile's back to branch, dramatic splash, dynamic motion capture",
      "Safe on tree: monkey sitting amidst jamun fruits, triumphant, warm golden sunset, calm river background"
    ]
  },
  {
    "id": "courage-1",
    "title": "The Brave Potter's Son",
    "theme": "courage",
    "content": "In a small village, a young potter's son discovered a fire threatening the granary at night. Everyone was asleep, but he ran through the smoke and carried buckets of water tirelessly until the fire died out. His courage saved the village's food supply and taught everyone that even the smallest hands can achieve great deeds when fear is ignored.",
    "scenes": [
      "Night village: moonlit huts with smoke rising from granary, quiet tension, flickering shadows",
      "The son grabs buckets: close-up on determined eyes, hands gripping clay buckets, dynamic firelight reflections",
      "Carrying water: motion blur as he runs, sparks flying, villagers half-awake in background",
      "Fire dying out: water splashing, flames retreating, dramatic light contrast",
      "Village sunrise: villagers cheering the exhausted boy, relief and pride on faces, soft warm morning glow"
    ]
  },
  {
    "id": "kindness-1",
    "title": "The Foolish Sage and the Hungry Crow",
    "theme": "kindness",
    "content": "A wise but absent-minded sage often forgot to eat. One day, he found a hungry crow pecking at grains near his hut. Instead of scolding it, he shared his meager food. The crow, grateful, later returned with a treasure hidden in the forest, showing that even small acts of kindness can bring unexpected blessings.",
    "scenes": [
      "Sage's hut: humble interior, soft morning light, sage arranging a few grains in a clay bowl",
      "Crow pecking: close-up of black feathers glinting, tiny grains scattered around, curious eyes",
      "Sharing food: sage extending bowl to crow, warm tones, gentle expressions",
      "Crow returning: dramatic forest backdrop, crow dropping a sparkling treasure near sage",
      "Resolution: sage amazed, villagers peeking curiously, sunlight highlighting newfound fortune"
    ]
  },
  {
    "id": "justice-1",
    "title": "The Clever Washerman",
    "theme": "justice",
    "content": "A dishonest rich man claimed that a poor washerman had stolen his gold coins. The village elders were about to punish the washerman, but he suggested testing the claim: they should weigh the washerman and the coins together, then separately. By clever reasoning, he revealed the man had hidden the coins in his own pockets. The washerman’s calm intelligence exposed injustice and taught the value of fairness and wit.",
    "scenes": [
      "Village square: elders and villagers gathered, tense expressions, sun casting long shadows",
      "Rich man accusing: proud stance, coins glittering, dramatic light highlighting greed",
      "Washerman's suggestion: calm and composed, pointing at scale, everyone listening, focus on reasoning",
      "Revealing deception: coins falling from rich man's pockets, shocked faces, dynamic movement",
      "Resolution: villagers cheering, washerman respected, bright balanced daylight emphasizing fairness"
    ]
  }
]



    
    processed_stories = []
    
    for story in stories:
        print(f"\n=== Processing Story: {story['title']} ===")
        
        try:
            # Generate images
            image_paths = generate_story_images(story)
            
            # Generate audio
            narration_text = story["content"].replace('\n', ' ').strip()
            audio_path = generate_audio_gtts(narration_text, f"{story['id']}_narration")
            
            # Generate video
            video_path = create_story_video(image_paths, audio_path, story['id'])
            
            # Generate subtitles
            audio_clip = AudioFileClip(str(audio_path))
            subtitles = generate_subtitles(story['content'], audio_clip.duration)
            
            # Create web data
            web_data = {
                "id": story['id'],
                "title": story['title'],
                "theme": story['id'].split('-')[0],
                "content": story['content'],
                "images": [f"/generated_content/images/{Path(p).name}" for p in image_paths],
                "audioUrl": f"/generated_content/audio/{audio_path.name}",
                "videoUrl": f"/generated_content/videos/{video_path}" if video_path else None,
                "subtitles": subtitles
            }
            
            processed_stories.append(web_data)
            print(f"✅ Successfully processed: {story['title']}")
            
        except Exception as e:
            print(f"❌ Error processing {story['title']}: {str(e)}")
    
    # Save all processed stories
    all_stories_file = OUTPUT_DIR / "all_stories_data.json"

# Load existing stories if file exists
    if all_stories_file.exists():
      with open(all_stories_file, 'r') as f:
        existing_stories = json.load(f)
    else:
      existing_stories = []

# Append new stories
    all_stories_combined = existing_stories + processed_stories

# Save back to JSON
    with open(all_stories_file, 'w') as f:
      json.dump(all_stories_combined, f, indent=2)

    
    print(f"\n🎉 Batch processing complete! Data saved to: {all_stories_file}")
    return processed_stories

# Uncomment to process all stories
processed_stories = process_all_stories()


=== Processing Story: The Monkey and the Crocodile ===


KeyboardInterrupt: 

In [8]:
# Function to process all 10 stories
def process_all_stories():
    """Process all stories in the dataset"""
    
    # You can define all 10 stories here or load from external file
    stories = [
  {
    "id": "wisdom-2",
    "title": "The Talkative Tortoise",
    "theme": "wisdom",
    "content": "A tortoise, slow but wise, loved to talk too much. One day, he invited birds to take him across a river. As they flew, he kept chatting and fell into the water. This story teaches that wisdom is not just knowledge, but knowing when to speak and when to stay silent.",
    "scenes": [
      "Riverbank: tortoise speaking animatedly to colorful birds, sunlight sparkling on water",
      "Takeoff: birds carrying tortoise, wings spread wide, river glistening below",
      "Tortoise talking: close-up of tortoise mouth open, birds struggling to balance",
      "Fall into water: splash, surprised expression, water droplets in motion",
      "Lesson: tortoise sitting on riverbank, thoughtful, calm waters reflecting morning sky"
    ]
  },
  {
    "id": "courage-2",
    "title": "The Fearless Tiger Cub",
    "theme": "courage",
    "content": "A young tiger cub saw a hunter trapping animals in the forest. Despite his size, he boldly confronted the hunter, scaring him away and freeing the trapped animals. The story highlights that courage comes in all sizes and sometimes the smallest actions make the biggest difference.",
    "scenes": [
      "Forest clearing: tiger cub peeking from bushes, tension in shadows",
      "Hunter setting trap: close-up on tools and cage, forest birds flying away",
      "Confrontation: cub growling, fur bristling, hunter startled",
      "Animals freed: rabbits, deer escaping, cub watching proudly",
      "Forest safe: cub sitting atop a rock, sun rays through trees, animals grazing peacefully"
    ]
  },
  {
    "id": "kindness-2",
    "title": "The Generous Banyan Tree",
    "theme": "kindness",
    "content": "A traveler resting under a banyan tree was offered shade, fruits, and shelter by the spirit of the tree. In return for this kindness, he shared stories and laughter, enriching the tree’s eternal life. It shows that kindness is a two-way gift.",
    "scenes": [
      "Traveler arriving: dusty road, sun beating down, banyan tree casting wide shade",
      "Tree offering fruits: branches lowering gently, fruits glowing, warm light",
      "Sharing stories: traveler sitting, animated gestures, soft wind rustling leaves",
      "Evening glow: tree's spirit smiling, golden sunset light",
      "Parting: traveler walking away, tree standing tall, forest in calm evening hues"
    ]
  },
  {
    "id": "justice-2",
    "title": "The Honest Woodcutter",
    "theme": "justice",
    "content": "A woodcutter accidentally dropped his axe into a river. A river spirit offered him a golden and a silver axe, but he honestly claimed only his own. Pleased, the spirit rewarded his honesty. The story emphasizes that fairness and truthfulness are always recognized.",
    "scenes": [
      "Forest river: woodcutter chopping wood, axe slipping into water",
      "River spirit appearing: shimmering water, ethereal glow",
      "Golden and silver axes: axes floating on water, woodcutter hesitating",
      "Honest claim: close-up on woodcutter speaking truth, spirit nodding",
      "Reward: woodcutter receiving all axes, joyful smile, sun sparkling on river"
    ]
  }
]




    
    processed_stories = []
    
    for story in stories:
        print(f"\n=== Processing Story: {story['title']} ===")
        
        try:
            # Generate images
            image_paths = generate_story_images(story)
            
            # Generate audio
            narration_text = story["content"].replace('\n', ' ').strip()
            audio_path = generate_audio_gtts(narration_text, f"{story['id']}_narration")
            
            # Generate video
            video_path = create_story_video(image_paths, audio_path, story['id'])
            
            # Generate subtitles
            audio_clip = AudioFileClip(str(audio_path))
            subtitles = generate_subtitles(story['content'], audio_clip.duration)
            
            # Create web data
            web_data = {
                "id": story['id'],
                "title": story['title'],
                "theme": story['id'].split('-')[0],
                "content": story['content'],
                "images": [f"public/generated_content/images/{Path(p).name}" for p in image_paths],
                "audioUrl": f"public/generated_content/audio/{audio_path.name}",
                "videoUrl": f"public/generated_content/videos/{video_path}" if video_path else None,
                "subtitles": subtitles
            }
            
            processed_stories.append(web_data)
            print(f"✅ Successfully processed: {story['title']}")
            
        except Exception as e:
            print(f"❌ Error processing {story['title']}: {str(e)}")
    
    # Save all processed stories
    all_stories_file = OUTPUT_DIR / "all_stories_data.json"

# Load existing stories if file exists
    if all_stories_file.exists():
      with open(all_stories_file, 'r') as f:
        existing_stories = json.load(f)
    else:
      existing_stories = []

# Append new stories
    all_stories_combined = existing_stories + processed_stories

# Save back to JSON
    with open(all_stories_file, 'w') as f:
      json.dump(all_stories_combined, f, indent=2)

    
    print(f"\n🎉 Batch processing complete! Data saved to: {all_stories_file}")
    return processed_stories

# Uncomment to process all stories
processed_stories = process_all_stories()


=== Processing Story: The Talkative Tortoise ===
API Test - Status Code: 404
⚠️ Unexpected response: Not Found

🎨 Generating image 1/5
Attempting to generate: Riverbank: tortoise speaking animatedly to colorfu... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\wisdom-2_scene_01.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 2/5
Attempting to generate: Takeoff: birds carrying tortoise, wings spread wid... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\wisdom-2_scene_02.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 3/5
Attempting to generate: Tortoise talking: close-up of tortoise mouth open,... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\wisdom-2_scene_03.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 4/5
Attempting to generate: Fall into water: splash, surprised expression

MoviePy - Done.
Moviepy - Writing video public\generated_content\videos\wisdom-2_video.mp4



Moviepy - Done !
Moviepy - video ready public\generated_content\videos\wisdom-2_video.mp4
✅ Successfully processed: The Talkative Tortoise

=== Processing Story: The Fearless Tiger Cub ===
API Test - Status Code: 404
⚠️ Unexpected response: Not Found

🎨 Generating image 1/5
Attempting to generate: Forest clearing: tiger cub peeking from bushes, te... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\courage-2_scene_01.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 2/5
Attempting to generate: Hunter setting trap: close-up on tools and cage, f... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\courage-2_scene_02.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 3/5
Attempting to generate: Confrontation: cub growling, fur bristling, hunter... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\courage-2_

MoviePy - Done.
Moviepy - Writing video public\generated_content\videos\courage-2_video.mp4



Moviepy - Done !
Moviepy - video ready public\generated_content\videos\courage-2_video.mp4
✅ Successfully processed: The Fearless Tiger Cub

=== Processing Story: The Generous Banyan Tree ===
API Test - Status Code: 404
⚠️ Unexpected response: Not Found

🎨 Generating image 1/5
Attempting to generate: Traveler arriving: dusty road, sun beating down, b... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\kindness-2_scene_01.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 2/5
Attempting to generate: Tree offering fruits: branches lowering gently, fr... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\kindness-2_scene_02.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 3/5
Attempting to generate: Sharing stories: traveler sitting, animated gestur... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\kindn

MoviePy - Done.
Moviepy - Writing video public\generated_content\videos\kindness-2_video.mp4



Moviepy - Done !
Moviepy - video ready public\generated_content\videos\kindness-2_video.mp4
✅ Successfully processed: The Generous Banyan Tree

=== Processing Story: The Honest Woodcutter ===
API Test - Status Code: 404
⚠️ Unexpected response: Not Found

🎨 Generating image 1/5
Attempting to generate: Forest river: woodcutter chopping wood, axe slippi... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\justice-2_scene_01.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 2/5
Attempting to generate: River spirit appearing: shimmering water, ethereal... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\justice-2_scene_02.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 3/5
Attempting to generate: Golden and silver axes: axes floating on water, wo... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\justice

MoviePy - Done.
Moviepy - Writing video public\generated_content\videos\justice-2_video.mp4



Moviepy - Done !
Moviepy - video ready public\generated_content\videos\justice-2_video.mp4
✅ Successfully processed: The Honest Woodcutter

🎉 Batch processing complete! Data saved to: public\generated_content\all_stories_data.json


In [8]:
# Function to process all 10 stories
def process_all_stories():
    """Process all stories in the dataset"""
    
    # You can define all 10 stories here or load from external file
    stories = [
  {
    "id": "wisdom-3",
    "title": "Birbal and the Silent Answer",
    "theme": "wisdom",
    "content": "One day, Emperor Akbar asked Birbal a puzzling question in court. Instead of replying with words, Birbal drew a single circle on the ground. When Akbar pressed further, Birbal explained that the circle meant 'life comes full circle, and truth always returns to itself.' The courtiers were stunned, and Akbar realized that sometimes silence and symbols speak more wisely than long speeches.",
    "scenes": [
      "Mughal court: Emperor Akbar seated on ornate throne, marble arches, courtiers in turbans and angarkhas, intricate carpets on floor",
      "Akbar leaning forward: royal attire with jewel-encrusted turban, Birbal standing calmly before him",
      "Birbal kneeling: drawing a circle on polished court floor with chalk, courtiers watching with curiosity",
      "Courtiers whispering: colorful turbans, silk robes, expressions of confusion",
      "Akbar smiling: golden light streaming through carved jharokha windows, court filled with awe"
    ]
  },
  {
    "id": "courage-3",
    "title": "The Girl Who Faced the Dacoits",
    "theme": "courage",
    "content": "In a rural village, a young girl overheard dacoits planning to loot her home at night. Instead of running, she lit torches around the fields and beat a drum loudly, making it seem like the whole village was awake and armed. The dacoits, frightened, fled into the forest. Her fearless courage saved her people.",
    "scenes": [
      "North Indian village: mud huts with thatched roofs under moonlight, girl hiding near a bamboo fence in ghagra-choli",
      "Preparing torches: girl tying cloth to sticks, lighting with clay lamp, fire illuminating her determined face",
      "Open field: girl in traditional dress beating a dhol drum, echoing across dark fields, torches burning in background",
      "Dacoits in turbans and dhotis: startled expressions, shadows retreating into dense forest",
      "Morning: villagers in colorful attire gathering around girl, sunrise casting warm golden light"
    ]
  },
  {
    "id": "kindness-3",
    "title": "The King and the Old Woman’s Roti",
    "theme": "kindness",
    "content": "A wandering king disguised as a traveler stopped by a poor old woman’s hut. She welcomed him warmly and offered half of her only roti. Touched by her generosity, the king later revealed his identity and rewarded her with wealth, declaring that her true treasure was her kind heart.",
    "scenes": [
      "Simple village hut: clay walls, cow dung plastered floor, old woman in cotton saree cooking on earthen chulha",
      "Traveler arrives: disguised king in plain dhoti-kurta with dusty turban, holding a walking stick",
      "Sharing meal: woman offering half of a bajra roti on a brass plate, warm firelight",
      "Revelation: king removing turban, golden ornaments glimmering, old woman astonished",
      "Reward: villagers in traditional attire watching as king gifts silken cloth, grains, and coins, morning sun brightening the scene"
    ]
  },
  {
    "id": "justice-3",
    "title": "The Honest Guard and the Missing Jewels",
    "theme": "justice",
    "content": "A rich landlord accused his night guard of stealing jewels that had gone missing. The guard swore he was innocent, but no one believed him. The village judge gave each suspect a bamboo stick and said, 'By morning, the thief’s stick will grow longer.' Fearful, the real thief cut a piece off his stick at night to hide his guilt. In the morning, the judge saw the shortened stick and exposed the true culprit, clearing the guard’s name.",
    "scenes": [
      "Courtyard of a haveli: wealthy landlord in silk angarkha, moustache curled, accusing guard in simple dhoti-kurta",
      "Village panchayat: judge seated on charpai under banyan tree, villagers in cotton turbans gathered around",
      "Judge handing bamboo sticks: each suspect holding a stick, clay pots and bullock cart in background",
      "Night scene: guilty man in dhoti secretly cutting stick with sickle, lantern light casting shadows",
      "Morning assembly: suspects lined up, judge examining sticks, villagers cheering as thief is exposed, guard relieved"
    ]
  }
]





    
    processed_stories = []
    
    for story in stories:
        print(f"\n=== Processing Story: {story['title']} ===")
        
        try:
            # Generate images
            image_paths = generate_story_images(story)
            
            # Generate audio
            narration_text = story["content"].replace('\n', ' ').strip()
            audio_path = generate_audio_gtts(narration_text, f"{story['id']}_narration")
            
            # Generate video
            video_path = create_story_video(image_paths, audio_path, story['id'])
            
            # Generate subtitles
            audio_clip = AudioFileClip(str(audio_path))
            subtitles = generate_subtitles(story['content'], audio_clip.duration)
            
            # Create web data
            web_data = {
                "id": story['id'],
                "title": story['title'],
                "theme": story['id'].split('-')[0],
                "content": story['content'],
                "images": [f"public/generated_content/images/{Path(p).name}" for p in image_paths],
                "audioUrl": f"public/generated_content/audio/{audio_path.name}",
                "videoUrl": f"public/generated_content/videos/{video_path}" if video_path else None,
                "subtitles": subtitles
            }
            
            processed_stories.append(web_data)
            print(f"✅ Successfully processed: {story['title']}")
            
        except Exception as e:
            print(f"❌ Error processing {story['title']}: {str(e)}")
    
    # Save all processed stories
    all_stories_file = OUTPUT_DIR / "all_stories_data.json"

# Load existing stories if file exists
    if all_stories_file.exists():
      with open(all_stories_file, 'r') as f:
        existing_stories = json.load(f)
    else:
      existing_stories = []

# Append new stories
    all_stories_combined = existing_stories + processed_stories

# Save back to JSON
    with open(all_stories_file, 'w') as f:
      json.dump(all_stories_combined, f, indent=2)

    
    print(f"\n🎉 Batch processing complete! Data saved to: {all_stories_file}")
    return processed_stories

# Uncomment to process all stories
processed_stories = process_all_stories()


=== Processing Story: Birbal and the Silent Answer ===
API Test - Status Code: 404
⚠️ Unexpected response: Not Found

🎨 Generating image 1/5
Attempting to generate: Mughal court: Emperor Akbar seated on ornate thron... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\wisdom-3_scene_01.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 2/5
Attempting to generate: Akbar leaning forward: royal attire with jewel-enc... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\wisdom-3_scene_02.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 3/5
Attempting to generate: Birbal kneeling: drawing a circle on polished cour... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\wisdom-3_scene_03.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 4/5
Attempting to generate: Courtiers whispering: colorful turbans,

MoviePy - Done.
Moviepy - Writing video public\generated_content\videos\wisdom-3_video.mp4



Moviepy - Done !
Moviepy - video ready public\generated_content\videos\wisdom-3_video.mp4
✅ Successfully processed: Birbal and the Silent Answer

=== Processing Story: The Girl Who Faced the Dacoits ===
API Test - Status Code: 404
⚠️ Unexpected response: Not Found

🎨 Generating image 1/5
Attempting to generate: North Indian village: mud huts with thatched roofs... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\courage-3_scene_01.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 2/5
Attempting to generate: Preparing torches: girl tying cloth to sticks, lig... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\courage-3_scene_02.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 3/5
Attempting to generate: Open field: girl in traditional dress beating a dh... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\ima

MoviePy - Done.
Moviepy - Writing video public\generated_content\videos\courage-3_video.mp4



Moviepy - Done !
Moviepy - video ready public\generated_content\videos\courage-3_video.mp4
✅ Successfully processed: The Girl Who Faced the Dacoits

=== Processing Story: The King and the Old Woman’s Roti ===
API Test - Status Code: 404
⚠️ Unexpected response: Not Found

🎨 Generating image 1/5
Attempting to generate: Simple village hut: clay walls, cow dung plastered... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\kindness-3_scene_01.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 2/5
Attempting to generate: Traveler arrives: disguised king in plain dhoti-ku... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\kindness-3_scene_02.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 3/5
Attempting to generate: Sharing meal: woman offering half of a bajra roti ... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_con

MoviePy - Done.
Moviepy - Writing video public\generated_content\videos\kindness-3_video.mp4



Moviepy - Done !
Moviepy - video ready public\generated_content\videos\kindness-3_video.mp4
✅ Successfully processed: The King and the Old Woman’s Roti

=== Processing Story: The Honest Guard and the Missing Jewels ===
API Test - Status Code: 404
⚠️ Unexpected response: Not Found

🎨 Generating image 1/5
Attempting to generate: Courtyard of a haveli: wealthy landlord in silk an... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\justice-3_scene_01.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 2/5
Attempting to generate: Village panchayat: judge seated on charpai under b... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\justice-3_scene_02.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 3/5
Attempting to generate: Judge handing bamboo sticks: each suspect holding ... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\gener

MoviePy - Done.
Moviepy - Writing video public\generated_content\videos\justice-3_video.mp4



Moviepy - Done !
Moviepy - video ready public\generated_content\videos\justice-3_video.mp4
✅ Successfully processed: The Honest Guard and the Missing Jewels

🎉 Batch processing complete! Data saved to: public\generated_content\all_stories_data.json


In [9]:
# Function to process all 10 stories
def process_all_stories():
    """Process all stories in the dataset"""
    
    # You can define all 10 stories here or load from external file
    stories = [
  {
    "id": "wisdom-4",
    "title": "Tenali Raman and the Greedy Brahmin",
    "theme": "wisdom",
    "content": "A Brahmin tried to trick King Krishnadevaraya into giving him more gold by claiming his rituals would fail without it. Tenali Raman, known for his wit, disguised himself as a holy man and told the Brahmin that greed would anger the gods. Exposed and embarrassed, the Brahmin admitted his lie. The king praised Tenali for showing that wisdom lies in exposing falsehood with cleverness.",
    "scenes": [
      "Vijayanagara court: granite-pillared mandapa, ornate canopy above throne, Krishnadevaraya in jeweled crown and silk angavastram, courtiers in dhotis and turbans",
      "Greedy Brahmin: tilak on forehead, rudraksha mala, palms joined in false humility, asking for gold",
      "Tenali as sage: saffron robes, wooden kamandalu, vibhuti lines on forehead, commanding presence",
      "Revelation: Brahmin sweating, glances of suspicion from nobles in pearl and silk attire",
      "Resolution: king laughing heartily, courtiers smiling, murals of gods painted on stone walls"
    ]
  },
  {
    "id": "courage-4",
    "title": "Abhimanyu in the Chakravyuha",
    "theme": "courage",
    "content": "In the Mahabharata, young Abhimanyu bravely entered the deadly Chakravyuha battle formation. Though he knew only how to enter and not how to exit, he fought fearlessly against mighty warriors. Surrounded and outnumbered, he fought valiantly till his last breath, remembered forever as a symbol of unmatched courage.",
    "scenes": [
      "Kurukshetra battlefield: red dust rising, rows of elephants with golden caparisons, war conches sounding",
      "Abhimanyu: youthful warrior in shining armor and crown, standing on decorated chariot, holding Gandiva bow",
      "Clash of warriors: arrows streaking, horses rearing, conches and drums echoing",
      "Encircled: Abhimanyu fending off multiple Kaurava generals in jeweled armor, divine glow around him",
      "Martyrdom: golden light bathing fallen Abhimanyu, broken chariot wheels, enemy warriors lowering weapons in awe"
    ]
  },
  {
    "id": "kindness-4",
    "title": "Shibi Raja and the Dove",
    "theme": "kindness",
    "content": "King Shibi, known for compassion, once sheltered a frightened dove fleeing from a hawk. The hawk demanded its prey, but Shibi offered his own flesh equal to the dove’s weight instead. Amazed, the gods revealed themselves, blessing the king for his unmatched kindness and sacrifice.",
    "scenes": [
      "Royal court in forest clearing: King Shibi seated on carved lion-throne under peepal tree, courtiers in fine cotton dhotis and jewelry",
      "Dove seeking refuge: trembling bird in king’s palm, hawk with spread wings circling above",
      "Offering flesh: king cutting his arm with ritual knife, priests aghast, blood falling into brass bowl",
      "Balance scale: traditional stone-and-rope scale, dove on one side, king’s flesh on the other",
      "Divine revelation: gods Indra and Agni emerging in radiant golden light, courtiers bowing to the miracle"
    ]
  },
  {
    "id": "justice-4",
    "title": "Yudhishthira’s Test of Truth",
    "theme": "justice",
    "content": "During their exile, the Pandavas were tested by Dharma in disguise. Yudhishthira, known for his fairness, answered riddles truthfully even when his brothers’ lives were at stake. His unshaken commitment to truth and justice revived them, proving that justice flows from righteousness and clarity of thought.",
    "scenes": [
      "Forest hermitage lake: lotus-filled waters, Pandavas lying unconscious on mossy ground",
      "Yaksha in disguise: radiant figure with deer-hide cloak, staff glowing faintly, forest aura divine",
      "Yudhishthira seated: folded hands, matted hair tied simply, dharmic calm on his face",
      "Answering riddles: golden light filtering through sal trees, Yudhishthira speaking firmly",
      "Revival: Pandavas awakening with divine glow, Dharma revealing true form, heavenly light over forest"
    ]
  }
]

    
    processed_stories = []
    
    for story in stories:
        print(f"\n=== Processing Story: {story['title']} ===")
        
        try:
            # Generate images
            image_paths = generate_story_images(story)
            
            # Generate audio
            narration_text = story["content"].replace('\n', ' ').strip()
            audio_path = generate_audio_gtts(narration_text, f"{story['id']}_narration")
            
            # Generate video
            video_path = create_story_video(image_paths, audio_path, story['id'])
            
            # Generate subtitles
            audio_clip = AudioFileClip(str(audio_path))
            subtitles = generate_subtitles(story['content'], audio_clip.duration)
            
            # Create web data
            web_data = {
                "id": story['id'],
                "title": story['title'],
                "theme": story['id'].split('-')[0],
                "content": story['content'],
                "images": [f"public/generated_content/images/{Path(p).name}" for p in image_paths],
                "audioUrl": f"public/generated_content/audio/{audio_path.name}",
                "videoUrl": f"public/generated_content/videos/{video_path}" if video_path else None,
                "subtitles": subtitles
            }
            
            processed_stories.append(web_data)
            print(f"✅ Successfully processed: {story['title']}")
            
        except Exception as e:
            print(f"❌ Error processing {story['title']}: {str(e)}")
    
    # Save all processed stories
    all_stories_file = OUTPUT_DIR / "all_stories_data.json"

# Load existing stories if file exists
    if all_stories_file.exists():
      with open(all_stories_file, 'r') as f:
        existing_stories = json.load(f)
    else:
      existing_stories = []

# Append new stories
    all_stories_combined = existing_stories + processed_stories

# Save back to JSON
    with open(all_stories_file, 'w') as f:
      json.dump(all_stories_combined, f, indent=2)

    
    print(f"\n🎉 Batch processing complete! Data saved to: {all_stories_file}")
    return processed_stories

# Uncomment to process all stories
processed_stories = process_all_stories()


=== Processing Story: Tenali Raman and the Greedy Brahmin ===
API Test - Status Code: 404
⚠️ Unexpected response: Not Found

🎨 Generating image 1/5
Attempting to generate: Vijayanagara court: granite-pillared mandapa, orna... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\wisdom-4_scene_01.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 2/5
Attempting to generate: Greedy Brahmin: tilak on forehead, rudraksha mala,... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\wisdom-4_scene_02.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 3/5
Attempting to generate: Tenali as sage: saffron robes, wooden kamandalu, v... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\wisdom-4_scene_03.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 4/5
Attempting to generate: Revelation: Brahmin sweating, gl

MoviePy - Done.
Moviepy - Writing video public\generated_content\videos\wisdom-4_video.mp4



Moviepy - Done !
Moviepy - video ready public\generated_content\videos\wisdom-4_video.mp4
✅ Successfully processed: Tenali Raman and the Greedy Brahmin

=== Processing Story: Abhimanyu in the Chakravyuha ===
API Test - Status Code: 404
⚠️ Unexpected response: Not Found

🎨 Generating image 1/5
Attempting to generate: Kurukshetra battlefield: red dust rising, rows of ... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\courage-4_scene_01.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 2/5
Attempting to generate: Abhimanyu: youthful warrior in shining armor and c... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\courage-4_scene_02.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 3/5
Attempting to generate: Clash of warriors: arrows streaking, horses rearin... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_conten

MoviePy - Done.
Moviepy - Writing video public\generated_content\videos\courage-4_video.mp4



Moviepy - Done !
Moviepy - video ready public\generated_content\videos\courage-4_video.mp4
✅ Successfully processed: Abhimanyu in the Chakravyuha

=== Processing Story: Shibi Raja and the Dove ===
API Test - Status Code: 404
⚠️ Unexpected response: Not Found

🎨 Generating image 1/5
Attempting to generate: Royal court in forest clearing: King Shibi seated ... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\kindness-4_scene_01.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 2/5
Attempting to generate: Dove seeking refuge: trembling bird in king’s palm... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\kindness-4_scene_02.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 3/5
Attempting to generate: Offering flesh: king cutting his arm with ritual k... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\

MoviePy - Done.
Moviepy - Writing video public\generated_content\videos\kindness-4_video.mp4



Moviepy - Done !
Moviepy - video ready public\generated_content\videos\kindness-4_video.mp4
✅ Successfully processed: Shibi Raja and the Dove

=== Processing Story: Yudhishthira’s Test of Truth ===
API Test - Status Code: 404
⚠️ Unexpected response: Not Found

🎨 Generating image 1/5
Attempting to generate: Forest hermitage lake: lotus-filled waters, Pandav... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\justice-4_scene_01.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 2/5
Attempting to generate: Yaksha in disguise: radiant figure with deer-hide ... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\justice-4_scene_02.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 3/5
Attempting to generate: Yudhishthira seated: folded hands, matted hair tie... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\j

MoviePy - Done.
Moviepy - Writing video public\generated_content\videos\justice-4_video.mp4



Moviepy - Done !
Moviepy - video ready public\generated_content\videos\justice-4_video.mp4
✅ Successfully processed: Yudhishthira’s Test of Truth

🎉 Batch processing complete! Data saved to: public\generated_content\all_stories_data.json


In [8]:
# Function to process all 10 stories
def process_all_stories():
    """Process all stories in the dataset"""
    
    # You can define all 10 stories here or load from external file
    stories = [
  {
    "id": "wisdom-5",
    "title": "Chanakya and the Single Strand of Hair",
    "theme": "wisdom",
    "content": "Chanakya once found a single strand of hair in his meal. Instead of eating, he scolded himself for negligence, declaring that a small carelessness today could lead to a great downfall tomorrow. His discipline taught that wisdom lies in attention to the smallest details.",
    "scenes": [
      "Ancient gurukul hut: palm-leaf manuscripts stacked, oil lamp flickering, Chanakya seated cross-legged on reed mat",
      "Meal on banana leaf: simple rice, lentils, vegetables, single hair visible on top",
      "Chanakya frowning: sharp eyes, long matted hair tied at back, finger raised in admonition",
      "Symbolic imagery: glowing hair strand magnified against dark backdrop, aura of warning",
      "Resolution: Chanakya meditating beneath neem tree, aura of discipline and wisdom surrounding him"
    ]
  },
  {
    "id": "courage-5",
    "title": "Nachiketa Before Yama",
    "theme": "courage",
    "content": "In the Katha Upanishad, young Nachiketa fearlessly approached Yama, the god of death, after being offered to him by his father in anger. Instead of cowering, Nachiketa asked Yama deep questions about life and the soul. His fearless pursuit of truth granted him divine wisdom, showing courage not just in battle, but in seeking eternal knowledge.",
    "scenes": [
      "Vedic yajna ground: father Vajashrava in white dhoti angrily pointing, fire altar blazing, Nachiketa walking away",
      "Path to underworld: glowing river Vaitarani, skeletal trees, Nachiketa calm and composed",
      "Yama’s palace: dark stone fortress with skull motifs, throne of iron, Yama with black complexion, crown, and mace",
      "Nachiketa questioning: child in plain cloth, folded hands, unwavering gaze, divine calm",
      "Blessing: Yama smiling, golden aura and sacred flames enveloping Nachiketa, cosmic backdrop with stars"
    ]
  },
  {
    "id": "kindness-5",
    "title": "Rantideva’s Hospitality",
    "theme": "kindness",
    "content": "King Rantideva, though reduced to poverty, received guests even when he had only a little food and water. He shared every last morsel, saying he desired only the well-being of all beings. The gods, moved by his kindness, appeared and blessed him with eternal glory.",
    "scenes": [
      "Simple hut: thatched roof, clay walls, Rantideva in plain dhoti seated on floor, clay vessels beside him",
      "Serving food: handful of rice and dal on banana leaves, weary travelers in torn garments receiving humbly",
      "Sharing water: small earthen pot tilted, last drops given with serene compassion",
      "Guests transforming: glowing divine forms of Vishnu and Brahma emerging from humble disguises",
      "Blessing: radiant light filling the hut, gods showering blessings, villagers in awe outside"
    ]
  },
  {
    "id": "justice-5",
    "title": "King Harishchandra’s Trial",
    "theme": "justice",
    "content": "King Harishchandra, famed for truthfulness, lost his kingdom and wealth but refused to abandon justice and honesty. Even when working as a cremation ground keeper, he demanded fees from his own queen when she came to perform their son’s rites. The gods, moved by his unwavering sense of justice, restored his kingdom and family.",
    "scenes": [
      "Palace renunciation: Harishchandra removing golden crown, courtiers shocked, marble pillars in background",
      "Cremation ground: smoky air, funeral pyres, Harishchandra in plain white dhoti collecting fees with sorrowful dignity",
      "Queen in grief: torn cotton saree, holding lifeless child, pleading with folded hands",
      "Divine revelation: gods Indra and Vishnu descending in radiant chariots, golden sky breaking through smoke",
      "Restoration: Harishchandra crowned again in golden durbar hall, joyous subjects in festive attire celebrating"
    ]
  }
]

    
    processed_stories = []
    
    for story in stories:
        print(f"\n=== Processing Story: {story['title']} ===")
        
        try:
            # Generate images
            image_paths = generate_story_images(story)
            
            # Generate audio
            narration_text = story["content"].replace('\n', ' ').strip()
            audio_path = generate_audio_gtts(narration_text, f"{story['id']}_narration")
            
            # Generate video
            video_path = create_story_video(image_paths, audio_path, story['id'])
            
            # Generate subtitles
            audio_clip = AudioFileClip(str(audio_path))
            subtitles = generate_subtitles(story['content'], audio_clip.duration)
            
            # Create web data
            web_data = {
                "id": story['id'],
                "title": story['title'],
                "theme": story['id'].split('-')[0],
                "content": story['content'],
                "images": [f"public/generated_content/images/{Path(p).name}" for p in image_paths],
                "audioUrl": f"public/generated_content/audio/{audio_path.name}",
                "videoUrl": f"public/generated_content/videos/{video_path}" if video_path else None,
                "subtitles": subtitles
            }
            
            processed_stories.append(web_data)
            print(f"✅ Successfully processed: {story['title']}")
            
        except Exception as e:
            print(f"❌ Error processing {story['title']}: {str(e)}")
    
    # Save all processed stories
    all_stories_file = OUTPUT_DIR / "all_stories_data.json"

# Load existing stories if file exists
    if all_stories_file.exists():
      with open(all_stories_file, 'r') as f:
        existing_stories = json.load(f)
    else:
      existing_stories = []

# Append new stories
    all_stories_combined = existing_stories + processed_stories

# Save back to JSON
    with open(all_stories_file, 'w') as f:
      json.dump(all_stories_combined, f, indent=2)

    
    print(f"\n🎉 Batch processing complete! Data saved to: {all_stories_file}")
    return processed_stories

# Uncomment to process all stories
processed_stories = process_all_stories()


=== Processing Story: Chanakya and the Single Strand of Hair ===
API Test - Status Code: 404
⚠️ Unexpected response: Not Found

🎨 Generating image 1/5
Attempting to generate: Ancient gurukul hut: palm-leaf manuscripts stacked... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\wisdom-5_scene_01.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 2/5
Attempting to generate: Meal on banana leaf: simple rice, lentils, vegetab... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\wisdom-5_scene_02.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 3/5
Attempting to generate: Chanakya frowning: sharp eyes, long matted hair ti... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\wisdom-5_scene_03.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 4/5
Attempting to generate: Symbolic imagery: glowing hai

MoviePy - Done.
Moviepy - Writing video public\generated_content\videos\wisdom-5_video.mp4



Moviepy - Done !
Moviepy - video ready public\generated_content\videos\wisdom-5_video.mp4
✅ Successfully processed: Chanakya and the Single Strand of Hair

=== Processing Story: Nachiketa Before Yama ===
API Test - Status Code: 404
⚠️ Unexpected response: Not Found

🎨 Generating image 1/5
Attempting to generate: Vedic yajna ground: father Vajashrava in white dho... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\courage-5_scene_01.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 2/5
Attempting to generate: Path to underworld: glowing river Vaitarani, skele... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\courage-5_scene_02.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 3/5
Attempting to generate: Yama’s palace: dark stone fortress with skull moti... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\im

MoviePy - Done.
Moviepy - Writing video public\generated_content\videos\courage-5_video.mp4



Moviepy - Done !
Moviepy - video ready public\generated_content\videos\courage-5_video.mp4
✅ Successfully processed: Nachiketa Before Yama

=== Processing Story: Rantideva’s Hospitality ===
API Test - Status Code: 404
⚠️ Unexpected response: Not Found

🎨 Generating image 1/5
Attempting to generate: Simple hut: thatched roof, clay walls, Rantideva i... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\kindness-5_scene_01.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 2/5
Attempting to generate: Serving food: handful of rice and dal on banana le... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\kindness-5_scene_02.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 3/5
Attempting to generate: Sharing water: small earthen pot tilted, last drop... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\kindnes

MoviePy - Done.
Moviepy - Writing video public\generated_content\videos\kindness-5_video.mp4



Moviepy - Done !
Moviepy - video ready public\generated_content\videos\kindness-5_video.mp4
✅ Successfully processed: Rantideva’s Hospitality

=== Processing Story: King Harishchandra’s Trial ===
API Test - Status Code: 404
⚠️ Unexpected response: Not Found

🎨 Generating image 1/5
Attempting to generate: Palace renunciation: Harishchandra removing golden... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\justice-5_scene_01.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 2/5
Attempting to generate: Cremation ground: smoky air, funeral pyres, Harish... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\justice-5_scene_02.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 3/5
Attempting to generate: Queen in grief: torn cotton saree, holding lifeles... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: public\generated_content\images\jus

MoviePy - Done.
Moviepy - Writing video public\generated_content\videos\justice-5_video.mp4



Moviepy - Done !
Moviepy - video ready public\generated_content\videos\justice-5_video.mp4
✅ Successfully processed: King Harishchandra’s Trial

🎉 Batch processing complete! Data saved to: public\generated_content\all_stories_data.json


## Instructions for Integration

1. **Copy generated files** to your web project's `public/generated_content/` folder
2. **Update story data** in your React app with the generated JSON
3. **Test the video playback** in your web application
4. **Adjust timing and effects** as needed for better user experience

### File Structure for Web App:
```
public/
├── generated_content/
│   ├── images/
│   │   ├── wisdom-1_scene_1.png
│   │   ├── wisdom-1_scene_2.png
│   │   └── ...
│   ├── audio/
│   │   └── wisdom-1_narration.mp3
│   └── videos/
│       └── wisdom-1_video.mp4
```